In [ ]:
from google.colab import drive 
drive.mount('/content/drive')


## Imports

In [ ]:
import tensorflow
import tensorflow as tf
from tensorflow import keras
from keras import utils
from tensorflow.keras.utils import plot_model,to_categorical
from tensorflow.keras.optimizers import Adam,SGD
from keras.models import Model,Sequential,load_model,model_from_json
from keras.layers import GlobalAveragePooling2D, Dense, BatchNormalization
from keras.losses import CategoricalCrossentropy
from tensorflow.keras.applications import (DenseNet121, DenseNet169, DenseNet201, 
      InceptionResNetV2, InceptionV3, MobileNet, MobileNetV2, ResNet101, ResNet152, ResNet50, ResNet50V2, VGG16, VGG19, Xception)
from sklearn import metrics
from keras.metrics import ( TrueNegatives,TruePositives, FalseNegatives, FalsePositives, AUC, Precision, Recall)
from sklearn.metrics import (classification_report, confusion_matrix, average_precision_score,
    precision_recall_curve, roc_auc_score, roc_curve, f1_score)
from sklearn.utils import class_weight
from sklearn import preprocessing
from tensorflow.python.client import device_lib
import cv2
from IPython.display import display
from PIL import Image
import seaborn as sns
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
from tqdm import tqdm_notebook as tq 
%matplotlib inline
import imutils
from imutils import paths
from tqdm import tqdm_notebook
import sklearn.metrics as metrics
import pandas as pd
import math  
import os
import numpy as np                                                                                                                                                                                                                                                                                                                                    
import seaborn
import timeit

## **Loading Image**

In [ ]:
img_folder='/content/drive/MyDrive/Covid-19/New Run/Data/Radio/train'
img_test = '/content/drive/MyDrive/Covid-19/New Run/Data/Radio/test'
img_valid = '/content/drive/MyDrive/Covid-19/New Run/Data/Radio/valid'
IMG_WIDTH=224
IMG_HEIGHT=224

In [ ]:
def Norm (pixels):
  # calculate global mean and standard deviation
  mean, std = pixels.mean(), pixels.std()
  # global standardization of pixels
  pixels = (pixels - mean) / std
  # confirm it had the desired effect
  return pixels

def create_dataset(img_folder):
   
    img_data_array=[]
    class_name=[]
   
    for dir1 in os.listdir(img_folder):
        for file in tq(os.listdir(os.path.join(img_folder, dir1))):
            image_path= os.path.join(img_folder, dir1,  file)
            image= cv2.imread(image_path)
            if len(image.shape)<3:
              image= cv2.cvtColor(image,cv2.COLOR_GRAY2RGB)
              image=cv2.resize(image, (IMG_HEIGHT, IMG_WIDTH),interpolation = cv2.INTER_AREA)
              image=np.array(image)
              image = image.astype('float32')
              image = Norm(image)
              img_data_array.append(image)
              class_name.append(dir1)
            else:
              image=cv2.resize(image, (IMG_HEIGHT, IMG_WIDTH),interpolation = cv2.INTER_AREA)
              image=np.array(image)
              image = image.astype('float32')
              image = Norm(image)
              img_data_array.append(image)
              class_name.append(dir1)
    return img_data_array, class_name
    
def data_aug (img_data, class_name):
  M = np.float32([[1,0,15],[0,1,15]])
  for i in range(len(img_data)):
    img = img_data[i]
    rows,cols,c = img.shape
    dst = cv2.warpAffine(img,M,(cols,rows))
    rot =  imutils.rotate(img, -10)
    img_data.append(dst)
    class_name.append(class_name[i])
    img_data.append(rot)
    class_name.append(class_name[i])
  return img_data, class_name

def data_prepro(img_data):
  image_data_pre=[]
  for i in range(len(img_data)):
    img = img_data[i]
    lab= cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    cl = clahe.apply(l)
    cl = cv2.cvtColor(cl, cv2.COLOR_BGR2RGB)
    image_data_pre.append(cl)
  return img_data_pre

In [ ]:
img_data, class_name =create_dataset(img_folder)
img_data1, class_name1 = data_aug (img_data, class_name)
target_dict={k: v for v, k in enumerate(np.unique(class_name1))}
target_dict
target_val=  [target_dict[class_name1[i]] for i in range(len(class_name1))]
trainX =np.array(img_data1, np.float32)
trainY = np.array(list(map(int,target_val)), np.float32)
print(trainX.shape)
print(trainY.shape)

In [ ]:
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
# img = mpimg.imread()
imgplot = plt.imshow(trainX[50])
plt.show()

In [ ]:
img_test_data1, class_test_name1 =create_dataset(img_test)
target_test_dict={k: v for v, k in enumerate(np.unique(class_test_name1))}
target_test_val=  [target_test_dict[class_test_name1[i]] for i in range(len(class_test_name1))]
testX =np.array(img_test_data1, np.float32)
testY = np.array(list(map(int,target_test_val)), np.float32)
print(testX.shape)
print(testY.shape)

In [ ]:
img_valid_data1, class_valid_name1 =create_dataset(img_valid)
target_valid_dict={k: v for v, k in enumerate(np.unique(class_valid_name1))}
target_valid_val=  [target_valid_dict[class_valid_name1[i]] for i in range(len(class_valid_name1))]
validX =np.array(img_valid_data1, np.float32)
validY = np.array(list(map(int,target_valid_val)), np.float32)
print(validX.shape)
print(validY.shape)

In [ ]:
%pylab inline
# img = mpimg.imread()
imgplot = plt.imshow(trainX[800])
plt.show()

In [ ]:
labels=['COVID','NORMAL','Viral Pneumonia']
train_Y = to_categorical(trainY,3)
valid_Y = to_categorical(validY,3)
plt.xticks(rotation=90)
plt.bar(x=labels, height=np.mean(train_Y, axis=0))
plt.title("Frequency of Each Class")
plt.show()

In [ ]:
def compute_class_freqs(labels):
    """
    Compute positive and negative frequences for each class.

    Args:
        labels (np.array): matrix of labels, size (num_examples, num_classes)
    Returns:
        positive_frequencies (np.array): array of positive frequences for each
                                         class, size (num_classes)
        negative_frequencies (np.array): array of negative frequences for each
                                         class, size (num_classes)
    """
    # total number of patients (rows)
    N = labels.shape[0]
    
    positive_frequencies = np.sum(labels == 0,axis=0) / N
    negative_frequencies = np.sum(labels == 1,axis=0) / N
    return positive_frequencies, negative_frequencies

In [ ]:
freq_pos, freq_neg = compute_class_freqs(train_Y)
freq_pos


In [ ]:
data = pd.DataFrame({"Class": labels, "Label": "Positive", "Value": freq_pos})
data = data.append([{"Class": labels[l], "Label": "Negative", "Value": v} for l,v in enumerate(freq_neg)], ignore_index=True)
plt.xticks(rotation=90)
f = sns.barplot(x="Class", y="Value", hue="Label" ,data=data)

In [ ]:
# Import and Initialize
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42)
# Flatten/Reshape Image Data to 1D Vector
X = np.reshape(trainX,(trainX.shape[0], trainX.shape[1] * trainX.shape[2] *trainX.shape[3]))

# Apply SMOTE
X_smote, y_smote = sm.fit_resample(X, trainY)

# Reshape upsampled data into original form for DNN model
X_train = X_smote.reshape(X_smote.shape[0], IMG_WIDTH, IMG_HEIGHT, 3) 

from tensorflow.keras.utils import to_categorical
y_smote1 = to_categorical(y_smote)
plt.xticks(rotation=90)
plt.bar(x=labels, height=np.mean(y_smote, axis=0))
plt.title("Frequency of Each Class")
plt.show()
print("Before SMOTE, counts of label '0':{} ".format(sum(trainY ==0)))
print("Before SMOTE, counts of label '1':{}".format(sum(trainY ==1)))
print("Before SMOTE, counts of label '2':{}\n".format(sum(trainY ==2)))
print("*****************************************************************")
print("After SMOTE, counts of label '0':{} ".format(sum(y_smote ==0)))
print("After SMOTE, counts of label '1':{}".format(sum(y_smote ==1)))
print("After SMOTE, counts of label '2':{}\n".format(sum(y_smote ==2)))

## **Trainig --DenseNet201**

In [ ]:
start = timeit.default_timer()

In [ ]:
model_name = 'DenseNet201_D1_WithDA'
# create the base pre-trained model
base_model=tf.keras.applications.DenseNet201(weights='imagenet', include_top=False)
x = base_model.output
# add a global spatial average pooling layer
x = GlobalAveragePooling2D()(x)
# and a logistic layer
predictions = Dense(3, activation="softmax")(x)
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
y_smote1 = to_categorical(y_smote)
hist= model.fit(X_train, y_smote1, epochs=20, batch_size=8, validation_data= (validX, valid_Y), shuffle = True, verbose=1)
hist_df = pd.DataFrame(hist.history) 

In [ ]:

stop = timeit.default_timer()
execution_time = stop - start
print("The model was trained in "+str(execution_time)) # It returns time in seconds


In [ ]:
hist_df = pd.DataFrame(hist.history) 
hist_csv_file = '/content/drive/MyDrive/Covid-19/New Run/SMOTE/History/'+model_name+'.csv'

with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

In [ ]:

plt.subplot(2,1,1)
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig('/content/drive/MyDrive/Covid-19/New Run/SMOTE/Accuracy/'+model_name+'.png', bbox_inches='tight', dpi=300)
plt.show()



In [ ]:
plt.subplot(2,1,2)
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.ylabel('Loss')
plt.xlabel('epoch')
plt.legend(['train', 'loss'], loc='upper left')
plt.savefig('/content/drive/MyDrive/Covid-19/New Run/SMOTE/Loss/'+model_name+'.png', bbox_inches='tight', dpi=300)
plt.show()

In [ ]:
labels = ['COVID-19',
          'NORMAL', 
          'Viral Pneumonia'
      ]
predicted_vals = model.predict(testX, steps = testX.shape[0])

In [ ]:
predicted_vals = np.argmax(predicted_vals, axis=1)
cm = confusion_matrix(testY, predicted_vals)
def plot_confusion_matrix(data, labels, output_filename):
    """Plot confusion matrix using heatmap.
 
    Args:
        data (list of list): List of lists with confusion matrix data.
        labels (list): Labels which will be plotted across x and y axis.
        output_filename (str): Path to output file.
 
    """
    plt.title("Confusion Matrix")
 
    seaborn.set(font_scale=0.5)
    ax = seaborn.heatmap(data, annot=True, fmt="d", cmap="YlGnBu", cbar_kws={'label': 'Scale'})
 
    ax.set_xticklabels(labels)
    ax.set_yticklabels(labels)
 
    ax.set(ylabel="True Label", xlabel="Predicted Label")
    plt.savefig(output_filename, bbox_inches='tight', dpi=300)
    seaborn.set(color_codes=True)
    plt.figure(1, figsize=(9, 6))

plot_confusion_matrix(cm,labels,'/content/drive/MyDrive/Covid-19/New Run/SMOTE/Confusion Matrix/'+model_name+'.png')



In [ ]:
# To plot a ROC curve and AUC score for multi-class classification:
# set plot figure size
fig, c_ax = plt.subplots(1,1, figsize = (12, 8))

def multiclass_roc_auc_score(y_test, y_pred, average="macro"):
    lb = preprocessing.LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    y_pred = lb.transform(y_pred)

    for (idx, c_label) in enumerate(labels): # all_labels: no of the labels, for ex. ['cat', 'dog', 'rat']
        fpr, tpr, thresholds = metrics.roc_curve(y_test[:,idx].astype(int), y_pred[:,idx])
        c_ax.plot(fpr, tpr, label = '%s (AUC:%0.2f)'  % (c_label, metrics.auc(fpr, tpr)))
    c_ax.plot(fpr, fpr, 'b-', label = 'Random Guessing')
    return metrics.roc_auc_score(y_test, y_pred, average=average)

# calling
# test_generator.reset() # resetting generator
multiclass_roc_auc_score(testY, predicted_vals)

## **Evaluation metrics**

In [ ]:
get_performance_metrics(test_y,preds_t, labels, acc=get_accuracy, prevalence=get_prevalence, sens=get_sensitivity, spec=get_specificity, ppv=get_ppv, npv=get_npv, auc=roc_auc_score,f1=f1_score)


In [ ]:
model_json = model.to_json()
with open('/content/drive/MyDrive/Covid-19/New Run/SMOTE/Model/'+ model_name + '.json', "w") as json_file:
    json_file.write(model_json)
model.save_weights('/content/drive/MyDrive/Covid-19/New Run/SMOTE/Model/'+ model_name + '.h5')
print("Saved model to disk")

# load json and create model
json_file = open('/content/drive/MyDrive/Covid-19/New Run/SMOTE/Model/'+ model_name + '.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights('/content/drive/MyDrive/Covid-19/New Run/SMOTE/Model/'+ model_name +'.h5')
print("Loaded model from disk")
config = model.get_config()
new_model = keras.Model.from_config(config)
